In [1]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import datetime
from pytz import timezone
from pyspark.sql.window import *

In [2]:
#current_date = (datetime.datetime.now().strftime("%Y%m%d")) 
current_date = '20210110'
current_date

'20210110'

In [3]:
#spark = SparkSession \
#    .builder \
#    .appName("Desafio Passei Direto") \
#    .getOrCreate()

spark = pyspark.sql.SparkSession.builder.appName("Desafio Passei Direto") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [4]:
pathJsonCourse = "./RAW_DATA/COUSERS/COURSES_DT="

pathJsonSession = "./RAW_DATA/SESSIONS/SESSIONS_DT="

pathJsonStudentFSub = "./RAW_DATA/STUDENT_FOLLOW_SUBJECT/STUDENT_FOLLOW_SUBJECT_DT="

pathJsonStudent = "./RAW_DATA/STUDENTS/STUDENTS_DT="

pathJsonSubjects = "./RAW_DATA/SUBJECTS/SUBJECTS_DT="

pathJsonSubscrip = "./RAW_DATA/SUBSCRIPTIONS/SUBSCRIPTIONS_DT="

pathJsonUniversities = "./RAW_DATA/UNIVERSITIES/UNIVERSITIES_DT="

In [5]:
parquetCourses = './REFINING_DATA/PARQUET/COUSERS/COURSES_DT='

parquetSession = './REFINING_DATA/PARQUET/SESSION/SESSION_DT='

parquetStudentFSub = './REFINING_DATA/PARQUET/STUDENT_FOLLOW_SUBJECT/STUDENT_FOLLOW_SUBJECT_DT='

parquetStudent = './REFINING_DATA/PARQUET/STUDENTS/STUDENTS_DT='

parquetSubjects = './REFINING_DATA/PARQUET/SUBJECTS/SUBJECTS_DT='

parquetSubscrip = './REFINING_DATA/PARQUET/SUBSCRIPTIONS/SUBSCRIPTIONS_DT='

parquetUniversities = './REFINING_DATA/PARQUET/UNIVERSITIES/UNIVERSITIES_DT='

In [6]:
deltaBronzeCourses = './REFINING_DATA/DELTA/BRONZE/COUSERS/COURSES_DT='

deltaBronzeSession = './REFINING_DATA/DELTA/BRONZE/SESSION/SESSION_DT='

deltaBronzeStudentFSub = './REFINING_DATA/DELTA/BRONZE/STUDENT_FOLLOW_SUBJECT/STUDENT_FOLLOW_SUBJECT_DT='

deltaBronzeStudent = './REFINING_DATA/DELTA/BRONZE/STUDENTS/STUDENTS_DT='

deltaBronzeSubjects = './REFINING_DATA/DELTA/BRONZE/SUBJECTS/SUBJECTS_DT='

deltaBronzeSubscrip = './REFINING_DATA/DELTA/BRONZE/SUBSCRIPTIONS/SUBSCRIPTIONS_DT='

deltaBronzeUniversities = './REFINING_DATA/DELTA/BRONZE/UNIVERSITIES/UNIVERSITIES_DT='

In [7]:
deltaSilverCourses = './REFINING_DATA/DELTA/SILVER/COUSERS/COURSES_DT='

deltaSilverSession = './REFINING_DATA/DELTA/SILVER/SESSION/SESSION_DT='

deltaSilverStudentFSub = './REFINING_DATA/DELTA/SILVER/STUDENT_FOLLOW_SUBJECT/STUDENT_FOLLOW_SUBJECT_DT='

deltaSilverStudent = './REFINING_DATA/DELTA/SILVER/STUDENTS/STUDENTS_DT='

deltaSilverSubjects = './REFINING_DATA/DELTA/SILVER/SUBJECTS/SUBJECTS_DT='

deltaSilverSubscrip = './REFINING_DATA/DELTA/SILVER/SUBSCRIPTIONS/SUBSCRIPTIONS_DT='

deltaSilverUniversities = './REFINING_DATA/DELTA/SILVER/UNIVERSITIES/UNIVERSITIES_DT='

In [8]:
pathGoldReview = './REFINING_DATA/DELTA/GOLD/GOLD_REVIEW_DT='

# Carregando Dataframe RawData Json

In [9]:
dfCourses = spark.read.json(pathJsonCourse+current_date)

dfSession = spark.read.json(pathJsonSession+current_date)

dfStudentFSub = spark.read.json(pathJsonStudentFSub+current_date)

dfStudent = spark.read.json(pathJsonStudent+current_date)

dfSubjects = spark.read.json(pathJsonSubjects+current_date)

dfSubscrip = spark.read.json(pathJsonSubscrip+current_date)

dfUniversities = spark.read.json(pathJsonUniversities+current_date)


# Conversão para Parquet

In [10]:
dfCourses.write.mode("overwrite").parquet(parquetCourses+current_date)

dfSession.write.mode("overwrite").parquet(parquetSession+current_date)

dfStudentFSub.write.mode("overwrite").parquet(parquetStudentFSub+current_date)

dfStudent.write.mode("overwrite").parquet(parquetStudent+current_date)

dfSubjects.write.mode("overwrite").parquet(parquetSubjects+current_date)

dfSubscrip.write.mode("overwrite").parquet(parquetSubscrip+current_date)

dfUniversities.write.mode("overwrite").parquet(parquetUniversities+current_date)

# Carregando dataframe com arquivo em Parquet

In [11]:
dfCoursesPaquet = spark.read.parquet(parquetCourses+current_date)

dfSessionPaquet = spark.read.parquet(parquetSession+current_date)

dfStudentFSubPaquet = spark.read.parquet(parquetStudentFSub+current_date)

dfStudentPaquet = spark.read.parquet(parquetStudent+current_date)

dfSubjectsPaquet = spark.read.parquet(parquetSubjects+current_date)

dfSubscripPaquet = spark.read.parquet(parquetSubscrip+current_date)

dfUniversitiesPaquet = spark.read.parquet(parquetUniversities+current_date)


# Transformando em DeltaLake primeira camada Bronze

In [12]:
dfCoursesPaquet.write.mode("overwrite").format("delta").save(deltaBronzeCourses+current_date)

dfSessionPaquet.write.mode("overwrite").format("delta").save(deltaBronzeSession+current_date)

dfStudentFSubPaquet.write.mode("overwrite").format("delta").save(deltaBronzeStudentFSub+current_date)

dfStudentPaquet.write.mode("overwrite").format("delta").save(deltaBronzeStudent+current_date)

dfSubjectsPaquet.write.mode("overwrite").format("delta").save(deltaBronzeSubjects+current_date)

dfSubscripPaquet.write.mode("overwrite").format("delta").save(deltaBronzeSubscrip+current_date)

dfUniversitiesPaquet.write.mode("overwrite").format("delta").save(deltaBronzeUniversities+current_date)

# Lendo Delta Bronze / Preparando a camada Silver 

In [13]:
dfCoursesBronze = spark.read.format("delta").load(deltaBronzeCourses+current_date)

dfSessionBronze = spark.read.format("delta").load(deltaBronzeSession+current_date)

dfStudentFSubBronze = spark.read.format("delta").load(deltaBronzeStudentFSub+current_date)

dfStudentBronze = spark.read.format("delta").load(deltaBronzeStudent+current_date)

dfSubjectsBronze = spark.read.format("delta").load(deltaBronzeSubjects+current_date)

dfSubscripBronze = spark.read.format("delta").load(deltaBronzeSubscrip+current_date)

dfUniversitiesBronze = spark.read.format("delta").load(deltaBronzeUniversities+current_date)

# Tratando realizando tratamento de linhas duplicadas

In [14]:
dfCoursesSilver = dfCoursesBronze.withColumn("row_num", row_number().over(Window.partitionBy("Name").orderBy(("Name")))).filter(col("row_num")==1).drop("row_num")

dfSessionSilver = dfSessionBronze.withColumn("row_num", row_number().over(Window.partitionBy("SessionStartTime", "StudentClient", "StudentId").orderBy(("StudentId")))).filter(col("row_num")==1).drop("row_num")

dfStudentFSubSilver = dfStudentFSubBronze.withColumn("row_num", row_number().over(Window.partitionBy("FollowDate", "FollowDate", "SubjectId").orderBy(("StudentId")))).filter(col("row_num")==1).drop("row_num")

dfStudentSilver = dfStudentBronze.withColumn("row_num", row_number().over(Window.partitionBy("id").orderBy(("Id")))).filter(col("row_num")==1).drop("row_num")

dfSubjectsSilver = dfSubjectsBronze.withColumn("row_num", row_number().over(Window.partitionBy("Name").orderBy(("Name")))).filter(col("row_num")==1).drop("row_num")

dfSubscripSilver = dfSubscripBronze.withColumn("row_num", row_number().over(Window.partitionBy("PaymentDate", "PlanType", "StudentId").orderBy(("StudentId")))).filter(col("row_num")==1).drop("row_num")

dfUniversitiesSilver = dfUniversitiesBronze.withColumn("row_num", row_number().over(Window.partitionBy("Name").orderBy(("Name")))).filter(col("row_num")==1).drop("row_num")

# Dicionando uma nova coluna com a data de inserção

In [15]:
dfCoursesSilver = dfCoursesSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfSessionSilver = dfSessionSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfStudentFSubSilver = dfStudentFSubSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfStudentSilver = dfStudentSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfSubjectsSilver = dfSubjectsSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfSubscripSilver = dfSubscripSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

dfUniversitiesSilver = dfUniversitiesSilver.withColumn('UPDATE_D_TIME',lit(datetime.datetime.now().astimezone(timezone('Etc/GMT+3')).strftime('%Y-%m-%d %H:%M:%S.%f')).cast("timestamp"))

# Escrevendo os dados tratados/refinados na camada silver do Delta Lake

In [16]:
dfCoursesSilver.write.format("delta").mode("overwrite").save(deltaSilverCourses+current_date)

dfSessionSilver.write.format("delta").mode("overwrite").save(deltaSilverSession+current_date)

dfStudentFSubSilver.write.format("delta").mode("overwrite").save(deltaSilverStudentFSub+current_date)

dfStudentSilver.write.format("delta").mode("overwrite").save(deltaSilverStudent+current_date)

dfSubjectsSilver.write.format("delta").mode("overwrite").save(deltaSilverSubjects+current_date)

dfSubscripSilver.write.format("delta").mode("overwrite").save(deltaSilverSubscrip+current_date)

dfUniversitiesSilver.write.format("delta").mode("overwrite").save(deltaSilverUniversities+current_date)

# Carregamento dos dados da camada Silver para criação da View Gold(Produção)

In [17]:
dfCoursesSilver = spark.read.format("delta").load(deltaSilverCourses+current_date).withColumnRenamed("Id",'CourseId').withColumnRenamed('Name','CourseName')

dfSessionSilver = spark.read.format("delta").load(deltaSilverSession+current_date)

dfStudentFSubSilver = spark.read.format("delta").load(deltaSilverStudentFSub+current_date)

dfStudentSilver = spark.read.format("delta").load(deltaSilverStudent+current_date)

dfSubjectsSilver = spark.read.format("delta").load(deltaSilverSubjects+current_date)

dfSubscripSilver = spark.read.format("delta").load(deltaSilverSubscrip+current_date)

dfUniversitiesSilver = spark.read.format("delta").load(deltaSilverUniversities+current_date).withColumnRenamed("Id",'UniversityId').withColumnRenamed('Name','UniversityName')

# Join dos dataframes em uma tabela única

In [18]:
dfGoldReviews = dfStudentSilver.join(dfCoursesSilver, on=["CourseId"])\
                                .join(dfUniversitiesSilver, on=["UniversityId"])\
                                .join(dfSubscripSilver, col("Id") == col("StudentId"))\
                                .join(dfSessionSilver, (col("Id") == dfSessionSilver.StudentId) & (dfStudentSilver.StudentClient == dfSessionSilver.StudentClient), how="inner")\
                                .join(dfStudentFSubSilver.join(dfSubjectsSilver, col("SubjectId") == col("Id")).select(dfSubjectsSilver.Id,"Name", "FollowDate").withColumnRenamed("Name",'SubjectName'))\
                                .select(dfStudentSilver.Id, "UniversityName", "CourseName", "RegisteredDate", "City", "State", "SessionStartTime","FollowDate","SubjectName", dfSessionSilver.StudentClient, "SignupSource", "PlanType", "PaymentDate")\
                                .withColumnRenamed("Id",'StudentId')

In [59]:
dfGoldReviews.select("StudentId").filter(col("StudentId") == '34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48').show(10, truncate=False)

+---------+
|StudentId|
+---------+
+---------+




# Escrevendo o dado tratado com join na camada Gold

In [19]:
#dfGoldReviews.write.format("delta").mode("overwrite").save(pathGoldReview+current_date)

# 2ª Parte

# Path arquivos Json Event Page View particionado

In [20]:
pahJsonEventPageView = 'RAW_DATA/EVENTPAGEVIEW/EVENTPAGEVIEW_DT='

In [21]:
dfEventPageView = spark.read.json(pahJsonEventPageView+current_date)

In [52]:
#dfEventPageView\
#.select("Last Accessed Url", "marketing_campaign", 'marketing_medium', "marketing_source", 'session_uuid')\
#.where(col("marketing_campaign").isNotNull()).show()


dfEventPageView\
.select("session_uuid").filter(col("session_uuid") =='56c9afc0946a2324b76ac9e590de1281f1e2a9187a638df8cc6c86030def6409').show()

+------------+
|session_uuid|
+------------+
+------------+



Infelimente não consegui achar um campo igual - Uma relação entre as duas bases - nos dois dataframes para fazer Join e também não obtive sucesso no comando union(na expectativa de unir/concatenar os dois dataframes, sem um campo em comum)

In [27]:
dfEventPageView.printSchema()

root
 |-- Last Accessed Url: string (nullable = true)
 |-- Page Category: string (nullable = true)
 |-- Page Category 1: string (nullable = true)
 |-- Page Category 2: string (nullable = true)
 |-- Page Category 3: string (nullable = true)
 |-- Page Name: string (nullable = true)
 |-- at: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- clv_total: long (nullable = true)
 |-- country: string (nullable = true)
 |-- custom_1: string (nullable = true)
 |-- custom_2: string (nullable = true)
 |-- custom_3: string (nullable = true)
 |-- custom_4: string (nullable = true)
 |-- device_new: boolean (nullable = true)
 |-- first-accessed-page: string (nullable = true)
 |-- install_uuid: string (nullable = true)
 |-- language: string (nullable = true)
 |-- library_ver: string (nullable = true)
 |-- marketing_campaign: string (nullable = true)
 |-- marketing_medium: string (nullable = true)
 |-- mark

In [73]:
dfEventPageView.select('studentId_clientType').show(truncate=False)

+------------------------------------------------------------------------+
|studentId_clientType                                                    |
+------------------------------------------------------------------------+
|34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48@Website|
|23fa8b361cc7ea61dea852d72eba9129f05981077e1f65f9241ddd62c2aa4b97@Website|
|14cc805c72b0eec6292916da59c9c45a8aa621a1138eb6959e49362844d47c40@Website|
|98774aeb803eb08e08020309fe8472af79033fbae26ea4282071f6555909599e@Webapp |
|122ac90aee4fc73660620456712cfea98d7bd720a1fc7193010d902848859912@Website|
|d799aecbb0a87cdd5412c68172189cc65d84644ac26e34e72bbc2cc07117eee8@Website|
|ca122c896b84fa32f750f95c9c163d3e1146c63e24f2ddf816467b0978c07d4c@Website|
|e1b0981e40fdef0c39b9e1442ca23aa10eceaee7f1013e98707b8b43416660aa@Website|
|859fa6b0af443a290eb97b19cc3e16eeea37cc60c05a5d35e6b880481a23e2c9@Webapp |
|344262de5df32d62ccd1592dfd9aa073a584d0fdd1f16455efe8bd82770b28c9@Website|
|249fc56e53f8e9218dad8667

In [62]:
dfEventPageView.select("session_uuid").show(truncate=False)

#.filter(col("uuid") =='56c9afc0946a2324b76ac9e590de1281f1e2a9187a638df8cc6c86030def6409').show()

+----------------------------------------+
|session_uuid                            |
+----------------------------------------+
|188031bec37fc43b737c2c49349076700ae89128|
|e3fe75ffce700fef07fd93e66312117ba008f046|
|e0841700c23ea9aaadf1104690e7d6e12a8a771a|
|318775acadbb0c72430ac2eeb05782589fbd1cf7|
|91bf1fbddb681569c914a8bb05fa0e260d9d92bd|
|d52b6c4881b52297521463e789d6721cb85e08b6|
|af9ab1ed319a1a8a62dc6e39638c7ee743340eb7|
|74c44cbf080da372d3f63ae61bd67ed006f1a2a6|
|0bf547c5ac802a0e0b3e28b8d5e012066fafa9d6|
|943ba5d00ab9607eb2979618e11c10ccec1a17dd|
|e79a1f3f047ebb8e427b0daae9cb69c513da97b6|
|72cfe273a58f954f9724839a4a49653ad1b1b0ea|
|ffdcc11a10bcc9489d59e7c31fac28af3cc7891b|
|ed965bcac88fad54b01cb8dad8cce2a2deb7ed4f|
|11a92cb247234ef8cad504cfc3817243a5726efa|
|bdba1aa54b7eccd61d3af87d38d6462f8ead4363|
|03c78e9f0944930b9c82f6319f371af3192606f6|
|4fd576c03ee8c335759f127c112f6da2c8b052b0|
|547f44b389a3144877bea971247d350cb447a8e7|
|599d273bb3fba56cf344d6a28656a63c48667073|
+----------

In [63]:
dfSessionSilver.printSchema()

root
 |-- SessionStartTime: string (nullable = true)
 |-- StudentClient: string (nullable = true)
 |-- StudentId: string (nullable = true)
 |-- UPDATE_D_TIME: timestamp (nullable = true)



In [65]:
dfSessionSilver.show()

+-------------------+-------------+--------------------+--------------------+
|   SessionStartTime|StudentClient|           StudentId|       UPDATE_D_TIME|
+-------------------+-------------+--------------------+--------------------+
|2017-11-01 00:04:27|      Android|f074f589b7f97a623...|2021-01-17 02:09:...|
|2017-11-01 00:21:18|      Android|540b82f637b0b1545...|2021-01-17 02:09:...|
|2017-11-01 01:28:56|      Website|fb3b99ecf669b03a0...|2021-01-17 02:09:...|
|2017-11-01 02:35:06|      Website|ab0545de63c1338ca...|2021-01-17 02:09:...|
|2017-11-01 03:15:14|          iOS|2d5713d4d43e29734...|2021-01-17 02:09:...|
|2017-11-01 08:00:27|      Website|0bda2af855c4bd084...|2021-01-17 02:09:...|
|2017-11-01 10:33:31|      Android|557d9b9337b221096...|2021-01-17 02:09:...|
|2017-11-01 10:52:29|      Website|533f7ba93a7ed5eb3...|2021-01-17 02:09:...|
|2017-11-01 11:46:03|      Website|2f77b382c77931871...|2021-01-17 02:09:...|
|2017-11-01 13:27:46|      Website|095cd3e757be276dd...|2021-01-

In [66]:
dfStudentSilver.printSchema()

root
 |-- City: string (nullable = true)
 |-- CourseId: long (nullable = true)
 |-- Id: string (nullable = true)
 |-- RegisteredDate: string (nullable = true)
 |-- SignupSource: string (nullable = true)
 |-- State: string (nullable = true)
 |-- StudentClient: string (nullable = true)
 |-- UniversityId: long (nullable = true)
 |-- UPDATE_D_TIME: timestamp (nullable = true)



In [82]:
dfStudentSilver.filter(col('Id') == '859fa6b0af443a290eb97b19cc3e16eeea37cc60c05a5d35e6b880481a23e2c9').show(truncate=False)

+----+--------+---+--------------+------------+-----+-------------+------------+-------------+
|City|CourseId|Id |RegisteredDate|SignupSource|State|StudentClient|UniversityId|UPDATE_D_TIME|
+----+--------+---+--------------+------------+-----+-------------+------------+-------------+
+----+--------+---+--------------+------------+-----+-------------+------------+-------------+



In [83]:
dfStudentSilver.select('Id').show(truncate=False)

+----------------------------------------------------------------+
|Id                                                              |
+----------------------------------------------------------------+
|00bc26f661c60bc4c4979feff681e7f8d11a8dac3222abeb74d163975ea8a3cc|
|01506b8c83f5904dff93fa9352faecbc27475dafab5e188e45d2522bb70dea37|
|020e02dea8d4161981569f69bd1785e032392dd4ff3c72ef5bdb92fb24a5daaa|
|021b9add9332d89d62fa0c49148850583cdbb2f99ed2e18414e2d1da4b28314e|
|03921d1e2e7dd284f493d2be97b70dcf84a7e1b1c8caf0f60d371a1f53dfd026|
|05793dfde79694ab9b70dd442e842a52f1ba834d9fdf7c928ad3a056c668b56e|
|058f4f62371b9c8260ae796dd4f2806ea15fca76bdf5f26a06e04031e7516154|
|05e1e243cf854530b23d31669e4ba84dd71f25c61888aa5bbf6750a3445360c1|
|070ce5e60ccab87066866854622ced32ece1806270a971d5c8e98a8f6eafac87|
|085ea7b2bd54b42a718e09a8c5686599ec65ac4cdcb96256dc54d0f1744b0af6|
|0949051ccf4252c0e91928be98204dc4eaf4bf43648d8a3e703cf7e8f6cf325c|
|0995ece82a5987427130492c1a888b5f2b3e638acd1c940b703681490d3ff

In [88]:
dfEventPageView.select('session_uuid').where("session_uuid like '%1226cfb070d4648bbc2fa1d80b3bddd981695e25393d37707ae66af52da1c6b0%'").show(truncate=False)

+------------+
|session_uuid|
+------------+
+------------+



In [85]:
dfEventPageView.select('studentId_clientType').where("studentId_clientType like '%00bc26f661c60bc4c4979feff681e7f8d11a8dac3222abeb74d163975ea8a3cc%'").show(truncate=False)

+--------------------+
|studentId_clientType|
+--------------------+
+--------------------+



In [ ]:
dfTeste = dfEventPageView.union(dfGoldReviews)

In [86]:
dfEventPageView.select('session_uuid').show(truncate=False)

+----------------------------------------+
|session_uuid                            |
+----------------------------------------+
|188031bec37fc43b737c2c49349076700ae89128|
|e3fe75ffce700fef07fd93e66312117ba008f046|
|e0841700c23ea9aaadf1104690e7d6e12a8a771a|
|318775acadbb0c72430ac2eeb05782589fbd1cf7|
|91bf1fbddb681569c914a8bb05fa0e260d9d92bd|
|d52b6c4881b52297521463e789d6721cb85e08b6|
|af9ab1ed319a1a8a62dc6e39638c7ee743340eb7|
|74c44cbf080da372d3f63ae61bd67ed006f1a2a6|
|0bf547c5ac802a0e0b3e28b8d5e012066fafa9d6|
|943ba5d00ab9607eb2979618e11c10ccec1a17dd|
|e79a1f3f047ebb8e427b0daae9cb69c513da97b6|
|72cfe273a58f954f9724839a4a49653ad1b1b0ea|
|ffdcc11a10bcc9489d59e7c31fac28af3cc7891b|
|ed965bcac88fad54b01cb8dad8cce2a2deb7ed4f|
|11a92cb247234ef8cad504cfc3817243a5726efa|
|bdba1aa54b7eccd61d3af87d38d6462f8ead4363|
|03c78e9f0944930b9c82f6319f371af3192606f6|
|4fd576c03ee8c335759f127c112f6da2c8b052b0|
|547f44b389a3144877bea971247d350cb447a8e7|
|599d273bb3fba56cf344d6a28656a63c48667073|
+----------